<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Start-to-Finish Example: Unit Testing `GiRaFFE_NRPy`: $\tilde{S}_i$ source term, $A_i$ gauge term, and $\psi^6 \Phi$

## Author: Patrick Nelson
### Formatting improvements courtesy Brandon Clark

## This module Validates the `Stilde_source_A_gauge_Phi_rhs` routine for `GiRaFFE`.

**Notebook Status:** <font color='red'><b>In Progress</b></font>

**Validation Notes:** This module will validate the routines in [Tutorial-GiRaFFE_NRPy-Stilde_source_A_gauge_Phi_rhs](Tutorial-GiRaFFE_NRPy-Stilde_source_A_gauge_Phi_rhs.ipynb).

### NRPy+ Source Code for this module: 
* [GiRaFFE_NRPy/Stilde_source_A_gauge_Phi_rhs.py](../../edit/in_progress/GiRaFFE_NRPy/Stilde_source_A_gauge_Phi_rhs.py) [\[**tutorial**\]](Tutorial-GiRaFFE_NRPy-Stilde_source_A_gauge_Phi_rhs.ipynb) Generates the driver to compute the magnetic field from the vector potential in arbitrary spactimes.

## Introduction:

This notebook validates the code that will add simpler terms to the RHSs of our evolved, conservative variables in `GiRaFFE_NRPy`. These terms require only basic finite-differencing for their derivatives, so the code to generate them is far simpler.

It is, in general, good coding practice to unit test functions individually to verify that they produce the expected and intended output. We will generate test data with arbitrarily-chosen analytic functions and calculate gridfunctions at the cell centers on a small numeric grid. We will then run the algorithms to compute the RHS terms numerically along with similar algorithms to compute them analytically, for comparison. We will then compare the data output by both routines to show convergence 


<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This notebook is organized as follows

1. [Step 1](#setup): Set up core functions and parameters for unit testing the algorithm
    1. [Step 1.a](#metric_expressions) Write expressions for the metric gridfunctions
    1. [Step 1.b](#mhd_expressions) Write expressions for the magnetohydrodynamic gridfunctions
    1. [Step 1.c](#ccodekernels) Generate C functions to calculate the gridfunctions
    1. [Step 1.d](#functions_to_test) Generate C code to numerically calculate RHSs
    1. [Step 1.e](#analytic_comparison) Generate C code to analytically calculate RHSs
    1. [Step 1.f](#free_parameters) Set free parameters in the code
1. [Step 2](#mainc): `Stilde_source_A_gauge_Phi_rhs_unit_test.c`: The Main C Code
    1. [Step 2.a](#compile_run): Compile and run the code
1. [Step 3](#convergence): Code validation: Verify that relative error in numerical solution converges to zero at the expected order
1. [Step 4](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='setup'></a>

# Step 1: Set up core functions and parameters for unit testing the algorithm \[Back to [top](#toc)\]
$$\label{setup}$$

We'll start by appending the relevant paths to `sys.path` so that we can access sympy modules in other places. Then, we'll import NRPy+ core functionality and set up a directory in which to carry out our test. 

In [ ]:
import shutil, os, sys           # Standard Python modules for multiplatform OS-level functions
# First, we'll add the parent directory to the list of directories Python will check for modules.
nrpy_dir_path = os.path.join("..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)
nrpy_dir_path = os.path.join("..","..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

from outputC import *            # NRPy+: Core C code output module
import finite_difference as fin  # NRPy+: Finite difference C code generation module
import NRPy_param_funcs as par   # NRPy+: Parameter interface
import grid as gri               # NRPy+: Functions having to do with numerical grids
import loop as lp                # NRPy+: Generate C code loops
import indexedexp as ixp         # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
import reference_metric as rfm   # NRPy+: Reference metric support
import cmdline_helper as cmd     # NRPy+: Multi-platform Python command-line interface

out_dir = "Validation/"
cmd.mkdir(out_dir)

thismodule = "Start_to_Finish-GiRaFFE_NRPy-Stilde_source_A_gauge_Phi_rhs"

# Set the finite-differencing order to 2
par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 2)


<a id='metric_expressions'></a>

## Step 1.a: Write expressions for the metric gridfunctions \[Back to [top](#toc)\]
$$\label{metric_expressions}$$

Now, we'll choose some functions with arbitrary forms to generate test data. We'll need to set ten gridfunctions, so expressions are being pulled from several previously written unit tests.

\begin{align}
\gamma_{xx} &= ax^3 + by^3 + cz^3 + dy^2 + ez^2 + f \\
\gamma_{yy} &= gx^3 + hy^3 + lz^3 + mx^2 + nz^2 + p \\
\gamma_{zz} &= px^3 + qy^3 + rz^3 + sx^2 + ty^2 + u. \\
\gamma_{xy} &= a \exp\left(-\left((x-b)^2+(y-c)^2+(z-d)^2\right)\right) \\
\gamma_{xz} &= f \exp\left(-\left((x-g)^2+(y-h)^2+(z-l)^2\right)\right) \\
\gamma_{yz} &= m \exp\left(-\left((x-n)^2+(y-o)^2+(z-p)^2\right)\right), \\
\beta^x &= \frac{2}{\pi} \arctan(ax + by + cz) \\
\beta^y &= \frac{2}{\pi} \arctan(bx + cy + az) \\
\beta^z &= \frac{2}{\pi} \arctan(cx + ay + bz) \\
\alpha &= 1 - \frac{1}{2+x^2+y^2+z^2} \\
\end{align}


<a id='mhd_expressions'></a>

## Step 1.b: Write expressions for the magnetohydrodynamic gridfunctions \[Back to [top](#toc)\]
$$\label{mhd_expressions}$$


<a id='ccodekernels'></a>

## Step 1.c: Generate C functions to calculate the gridfunctions \[Back to [top](#toc)\]
$$\label{ccodekernels}$$


<a id='functions_to_test'></a>

## Step 1.d: Generate C code to numerically calculate RHSs \[Back to [top](#toc)\]
$$\label{functions_to_test}$$


<a id='analytic_comparison'></a>

## Step 1.e: Generate C code to analytically calculate RHSs \[Back to [top](#toc)\]
$$\label{analytic_comparison}$$


<a id='free_parameters'></a>

## Step 1.f: Set free parameters in the code \[Back to [top](#toc)\]
$$\label{free_parameters}$$


<a id='mainc'></a>

# Step 2: `Stilde_source_A_gauge_Phi_rhs_unit_test.c`: The Main C Code \[Back to [top](#toc)\]
$$\label{mainc}$$


<a id='convergence'></a>

# Step 3: Code validation: Verify that relative error in numerical solution converges to zero at the expected order \[Back to [top](#toc)\]
$$\label{convergence}$$


<a id='latex_pdf_output'></a>

# Step 4: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$
